In [ ]:
"""
Created on Mon Aug 30 16:59 2021

This script is to produce Figure 1 and Figure 2

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import assess_param_funcs.useful_functions as uf
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import glob

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'


In [ ]:
run_list = ['OPM021','OPM018','OPM016','OPM006'] 
file_isf_list = []
TS_profile_mean_list = []
TS_profile_std_list = []
plume_list = []

for n,nemo_run in enumerate(run_list):
    
    print(nemo_run)
    
    # File mask
    inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf_list.append(file_isf)
    
    # Files with profiles
    inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
    file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
    if nemo_run == 'OPM006':
        file_TS_otherdepth = file_TS_orig.sel(Nisf=large_isf)
        file_TS = file_TS_otherdepth.interp({'depth': TS_profile_mean_list[0].depth})
    else:
        file_TS = file_TS_orig.sel(Nisf=large_isf)
    file_TS_mean = file_TS.mean('time')
    file_TS_std = file_TS.std('time')
    TS_profile_mean_list.append(file_TS_mean)
    TS_profile_std_list.append(file_TS_std)
    
    # Files with plume characteristics
    inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')
    plume_list.append(plume_charac)

# Merge them over dimension nemo_runs
file_isf_all = xr.concat(file_isf_list, dim='nemo_run')

TS_profile_mean = xr.concat(TS_profile_mean_list, dim='nemo_run')
TS_profile_std = xr.concat(TS_profile_std_list, dim='nemo_run')
plume_all =  xr.concat(plume_list, dim='nemo_run')

In [ ]:
# write out deepest grounding line
deep_GL_list = []
for kisf in tqdm(file_isf['Nisf']):
    deepest_GL = plume_all['zGL'].sel(option='simple').where(file_isf['ISF_mask']==kisf,drop=True).mean(['x','y'])
    deep_GL_list.append(deepest_GL)
deepest_GL_all = xr.concat(deep_GL_list, dim='Nisf')

Make a bit of order in the ice shelves

In [ ]:
region_list = []
for kisf in file_isf.Nisf:
    if file_isf['isf_name'].sel(Nisf=kisf).values in ['Ross','Nickerson','Sulzberger', 'Cook']:
        region_list.append('East and West Ross')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Filchner','Ronne','Filchner-Ronne']:
        region_list.append('Weddell')        
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Ekstrom','Nivl','Prince Harald','Riiser-Larsen','Fimbul','Roi Baudouin','Lazarev','Stancomb Brunt','Jelbart','Borchgrevink']:
        region_list.append('Dronning Maud Land')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Getz','Thwaites','Crosson','Dotson','Cosgrove','Pine Island']:
        region_list.append('Amundsen')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Venable','George VI','Abbot','Stange','Larsen C','Bach','Larsen D','Wilkins']:
        region_list.append('Peninsula and Bellinghausen')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Amery','Moscow Univ.','Tracy Tremenchus','Totten','West','Shackleton']:
        region_list.append('East Antarctica')
    else:
        print('Argh, help me, '+file_isf['isf_name'].sel(Nisf=kisf).values+' has no region assigned!')
# 6 regions
# regions = ['East and West Ross','Weddell','Dronning Maud Land','Amundsen','Peninsula and Bellinghausen','East Antarctica']
file_isf['region'] = xr.DataArray(data=region_list,dims='Nisf')

In [ ]:
file_isf['isf_name'] = file_isf['isf_name'].where(file_isf['isf_name'] != 'Ekstrom', np.array('Ekström', dtype=object))

PLOT FIGURE 1

In [ ]:
regions = ['Weddell','Peninsula and Bellinghausen','Amundsen','East and West Ross','East Antarctica','Dronning Maud Land']
colors = ['red','orange','brown','magenta','yellow','cyan','blue']

dom = 50

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0
for rr,reg in enumerate(regions):
    
    subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
    
    for kisf in tqdm(subset_isf.Nisf):

        mean_data = TS_profile_mean['theta_ocean'].sel(profile_domain=dom).sel(Nisf=kisf)
        std_data = TS_profile_std['theta_ocean'].sel(profile_domain=dom).sel(Nisf=kisf)
        
        ax[i] = f.add_subplot(6,6,i+1)
        
        for n, nrun in enumerate(mean_data.nemo_run):
            
            mean_run = mean_data.sel(nemo_run=nrun)
            std_run = std_data.sel(nemo_run=nrun)
            
            leg, = plt.plot(mean_run,-1*TS_profile_mean.depth, color=colors[n], linewidth=3)
            plt.fill_betweenx(-1*TS_profile_mean.depth, mean_run - std_run, mean_run + std_run, -1*TS_profile_mean.depth, alpha=0.2, color=colors[n])
            leg_hdl.append(leg)
            
            plt.axhline(y=-file_isf_all['front_bot_depth_max'].sel(nemo_run=n, Nisf=kisf), color='brown',linestyle='dotted')
            plt.axhline(y=-file_isf_all['front_bot_depth_avg'].sel(nemo_run=n, Nisf=kisf), color='chocolate',linestyle='dotted')
            if -file_isf_all['front_bot_depth_max'].sel(nemo_run=n, Nisf=kisf) < -1500:
                plt.axhline(y=-1500, color='k',linestyle='dotted')
            plt.axhline(y=deepest_GL_all.sel(nemo_run=n, Nisf=kisf), color='slategrey',linestyle='dotted')
        
        
        ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
        ax[i].set_xlim(-2.2,2.5)
        
        i = i+1
        

f.subplots_adjust(bottom=0.05, wspace=0.1)

f.legend(handles = leg_hdl , labels=['REALISTIC', 'COLDAMU', 'WARMROSS', 'HIGHGETZ'], loc='lower center', ncol=4)

f.tight_layout()
sns.despine()
f.savefig(plot_path+'Mean_T_profiles_'+str(dom)+'km.pdf')

PLOT FIGURE 2

In [ ]:
regions = ['Weddell','Peninsula and Bellinghausen','Amundsen','East and West Ross','East Antarctica','Dronning Maud Land']
colors = ['turquoise','deepskyblue','steelblue','blue','navy']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0
for rr,reg in enumerate(regions):
    
    subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
    
    for kisf in tqdm(subset_isf.Nisf):

        mean_data = TS_profile_mean['theta_ocean'].sel(nemo_run=0).sel(Nisf=kisf)
        std_data = TS_profile_std['theta_ocean'].sel(nemo_run=0).sel(Nisf=kisf)
        mean_data_cut = mean_data.where(np.isfinite(mean_data.sel(profile_domain=50)), drop=True)
        std_data_cut = std_data.where(np.isfinite(std_data.sel(profile_domain=50)), drop=True)

        
        ax[i] = f.add_subplot(6,6,i+1)
        
        for d, dom in enumerate(mean_data.profile_domain):
            
            mean_run = mean_data_cut.sel(profile_domain=dom)
            std_run = std_data_cut.sel(profile_domain=dom)
            
            leg, = plt.plot(mean_run,-1*mean_run.depth, color=colors[d], linewidth=3)
            plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[d])
            leg_hdl.append(leg)
        
            plt.axhline(y=-file_isf_all['front_bot_depth_max'].sel(nemo_run=n, Nisf=kisf), color='brown',linestyle='dotted')
            plt.axhline(y=-file_isf_all['front_bot_depth_avg'].sel(nemo_run=n, Nisf=kisf), color='chocolate',linestyle='dotted')
            if -file_isf_all['front_bot_depth_max'].sel(nemo_run=n, Nisf=kisf) < -1500:
                plt.axhline(y=-1500, color='k',linestyle='dotted')
            plt.axhline(y=deepest_GL_all.sel(nemo_run=0, Nisf=kisf), color='slategrey',linestyle='dotted')
        
        ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
        ax[i].set_xlim(-2.2,2.5)
        
        i = i+1
        

f.subplots_adjust(bottom=0.05, wspace=0.1)

f.legend(handles = leg_hdl , labels=[10, 25, 50, 100, 'offshore'], loc='lower center', ncol=5)

f.tight_layout()
sns.despine()
f.savefig(plot_path+'Mean_T_profiles_REALISTIC_cut.pdf')